In [3]:
!pip -q install streamlit
!pip -q install pyngrok


     |████████████████████████████████| 8.3 MB 26.8 MB/s 
     |████████████████████████████████| 4.3 MB 56.8 MB/s 
     |████████████████████████████████| 75 kB 5.8 MB/s 
     |████████████████████████████████| 111 kB 75.7 MB/s 
     |████████████████████████████████| 180 kB 60.1 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 124 kB 73.2 MB/s 
     |████████████████████████████████| 788 kB 56.4 MB/s 
     |████████████████████████████████| 370 kB 59.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.4.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.28.0 which is incompatib

In [1]:
!pip install opencv-python
!pip install easyocr

     |████████████████████████████████| 63.6 MB 35 kB/s 
     |████████████████████████████████| 37.1 MB 91 kB/s 


In [2]:
from pyngrok import ngrok

In [21]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import cv2
import easyocr
import requests
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from io import BytesIO
import urllib.request
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

#st.title('Predicting a Meme Dank or Not?')
html_temp = """
    <div style="background-color:grey;padding:10px">
    <h2 style="color:black;text-align:center;">Predicting a Meme Dank or Not? </h2>
    </div>"""
st.markdown(html_temp,unsafe_allow_html=True)

predict_meme = st.file_uploader("Select a File")
reader = easyocr.Reader(['en'],gpu = True)
tokenizer = pd.read_pickle('/content/tokenizer.pkl')
model = load_model('/content/drive/MyDrive/case_study_2/output/my_model.h5')

# Tokenize and lemmatize
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')


def preprocess(text):
    output=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            output.append(lemmatize_stemming(token))
            
    return output

if predict_meme is not None:
    query_meme = pd.read_csv(predict_meme)
    content = requests.get(query_meme['media'].values[0]).content
    image = Image.open(BytesIO(content))
    st.image(image,use_column_width=True)
    urllib.request.urlretrieve(query_meme.media.values[0],'image.jpg')
    output = reader.readtext('image.jpg',decoder = 'greedy', beamWidth= 5, batch_size = 1,
                            workers = 0, allowlist = None, blocklist = None, detail = 1,
                            rotation_info = None, paragraph = False, min_size = 20,
                            contrast_ths = 0.1,adjust_contrast = 0.5, filter_ths = 0.003,
                            text_threshold = 0.7, low_text = 0.4, link_threshold = 0.4,
                            canvas_size = 2560, mag_ratio = 1.,
                            slope_ths = 0.1 , ycenter_ths = 0.5, height_ths = 0.5,
                            width_ths = 0.5, y_ths = 0.5, x_ths = 1.0, add_margin = 0.1, output_format='standard')
    text = ""  
    for i in range(len(output)):
            text = text + " " +''.join(output[i][1])   
            text = text.strip()
    text = query_meme['title'].values[0] + ' ' + text
    processed_words = preprocess(text)
    processed_words = [word for word in processed_words if word not in stop] 
    memetext = ' '.join(processed_words)
    query_meme['Text'] = memetext
    sequences = tokenizer.texts_to_sequences(query_meme['Text'])
    sequences_matrix = pad_sequences(sequences,maxlen=225)
    response = requests.get(query_meme['media'].values[0],stream=True)
            #opening the image
    image= np.array(Image.open(response.raw))
    image= np.resize(image,(299,299,3))
    image = image.astype('float32')
    image /= 255
    image = np.expand_dims(image, axis=0)
    if st.button("Predict"):
        prediction = model.predict([sequences_matrix[:1],np.array(image[:1],np.float32)])
        st.write("Meme is :")
        if prediction<0.5:
            st.success('NOT DANK')
        else:
            st.success('Dank')
    

Overwriting app.py


In [22]:
!ngrok authtoken 1vaAzvxuZoTRjkNKSpUk282c9n1_2YZXiZuLEh4TAGMNgZBFN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [23]:
public_url = ngrok.connect(port='80')
print(public_url)
!streamlit run --server.port 80 app.py >/dev/null

NgrokTunnel: "http://a3c3-34-86-44-114.ngrok.io" -> "http://localhost:80"
2021-09-30 09:47:36.852 'pattern' package not found; tag filters are not available for English
2021-09-30 09:47:44.470067: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-30 09:47:44.477488: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-30 09:47:44.478142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-30 09:47:44.479144: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at 